**EJECUCIÓN LOCAL**. Este cuaderno está diseñado para ser descargado y ejecutado localmente. Si no se dispone de un entorno local Jupyter, la forma más inmediata de ejecución consiste en descargarse [JupyterLab Desktop](https://github.com/jupyterlab/jupyterlab-desktop#download). Este app se instala y se ejecuta directamente, 'con un doble-click'.

## Sobre este cuaderno

1. Recopila un grafo RDF mediante una consulta CONSTRUCT a Wikidata
2. Expone una forma de visualizarlo mediante el paquete [Jaal](https://github.com/imohitmayank/jaal)
3. Ejecuta conexiones REST API a una instancia local de GraphDB Desktop para (1) transferir el grafo y (2) para ejecutar consultas posteriores a esa base de datos orientada a grafos


## Datos, desde Wikidata
Recopilamos alguna información sobre museos localizados en España, con retorno de tipo grafo; es decir, como conjunto de tripletas (sujeto,predicado,objeto). Esta consulta de tipo CONSTRUCT ya ha sido utilizada en algún otro cuaderno de esta asignatura.

In [ ]:
# Comente o descomente conforme los paquetes estén instalados
!pip install sparqlwrapper rdflib 

In [1]:
from SPARQLWrapper import SPARQLWrapper, RDFXML, TURTLE
from rdflib import Graph, URIRef, Literal, RDF, RDFS

In [2]:
consulta = '''
CONSTRUCT {
  # Por el cuerpo del WHERE, los ?sujeto son museos en España.
  ?sujeto ?propD ?objeto ;
          rdfs:label ?sujetoLabel ;
          rdf:type ?sujetoClase .
   
  ?prop rdf:type wikibase:Property ;
        rdfs:label ?propLabel ;
        wikibase:directClaim ?propD ;
        wikibase:propertyType ?propTipoObjeto .
  
  ?objeto rdfs:label ?objetoLabel ;
              wdt:P31 ?objetoClase .              
}
WHERE {
  ?prop rdf:type wikibase:Property ;                 
            wikibase:directClaim ?propD ;             
            wikibase:propertyType ?propTipoObjeto. 
  
  ?sujeto wdt:P31/wdt:P279* wd:Q33506 ;              
          wdt:P31 ?sujetoClase ;                     
          wdt:P17 wd:Q29 ;                           
          ?propD ?objeto .                           
  
  OPTIONAL {                                         
    ?objeto wdt:P31 ?objetoClase .
  } .          
  
  # De cualquier ?item, este servicio proporciona su etiqueta en ?itemLabel 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "es","ca","eu","gl","en","fr". }
}
'''

In [3]:
servidor = "https://query.wikidata.org/sparql" 
cliente = SPARQLWrapper(servidor)
cliente.setReturnFormat(RDFXML)

cliente.setQuery(consulta)
resp_obj = cliente.query()

In [4]:
resp_grafo = resp_obj.convert()
resp_grafo_ttl = resp_grafo.serialize(format="turtle")
print(resp_grafo_ttl[:5000])

@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .
@prefix wikibase: <http://wikiba.se/ontology#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

wd:P1004 a wikibase:Property ;
    rdfs:label "identificador de lugar MusicBrainz"@es ;
    wikibase:directClaim wdt:P1004 ;
    wikibase:propertyType wikibase:ExternalId .

wd:P1005 a wikibase:Property ;
    rdfs:label "identificador PTBNP"@es ;
    wikibase:directClaim wdt:P1005 ;
    wikibase:propertyType wikibase:ExternalId .

wd:P101 a wikibase:Property ;
    rdfs:label "campo de trabajo"@es ;
    wikibase:directClaim wdt:P101 ;
    wikibase:propertyType wikibase:WikibaseItem .

wd:P1015 a wikibase:Property ;
    rdfs:label "identificador BIBSYS"@es ;
    wikibase:directClaim wdt:P1015 ;
    wikibase:propertyType wikibase:ExternalId .

wd:P1017 a wikibase:Property ;
    rd

### Datos reformateados como dataframe
Como los datos son tabulares, se presentan en un DataFrame, para facilitar su lectura.

In [ ]:
import pandas as pd

In [ ]:
pers_df = pd.DataFrame(pers)
display(pers_df)

print(f"{pers_df['nombre'][0]} nació en {pers_df['año_nacimiento'][0]} en {pers_df['lugar_nacimiento'][0]}")

In [ ]:
ejes_df = pd.DataFrame(pers_df['nombre'],'lugar_nacimiento',pers_df['lugar_nacimiento'])
display(ejes_df)

In [ ]:
# import
from jaal import Jaal
from dash import html
from jaal.datasets import load_got
# load the data
edge_df, node_df = load_got()
display(edge_df)
display(node_df)

In [ ]:
# init Jaal and run server
Jaal(edge_df, node_df).plot()

## Grafo
Con los datos de la tabla anterior se generan tripletas (\<sujeto\> \<predicado\> \<objeto\>) como las siguientes:
+ \<Juan\> \<año_de_nacimiento\> "2020"
+ \<Juan\> \<tiene_madre\> \<Ana\>

In [ ]:
from rdflib import Graph, Literal, URIRef, Namespace, RDF, RDFS

### Espacios de nombres

In [ ]:
EXR = Namespace("http://example.org/recursos#")
EXP = Namespace("http://example.org/propiedades#")

# Recursos citados desde la lista de diccionarios 'pers'
print(URIRef(EXR + pers[0]['nombre']))
print(URIRef(EXP.año_nacimiento))
print(Literal(pers[0]['año_nacimiento']))

### Generación del grafo

In [ ]:
g = Graph()

for p in pers:
    sujeto = URIRef(EXR + p['nombre'])
    for key, value in p.items():
        if key=='nombre':
            pass
        elif key=='año_nacimiento' :
            g.add( (sujeto, URIRef(EXP + key), Literal(value)) )
        else:
            g.add( (sujeto, URIRef(EXP + key), URIRef(EXR + value)) )

In [ ]:
# Serialización del grafo RDF creado, en formato Turtle.
g_ttl = g.serialize(format='turtle')
g_ttl

### Visualización del grafo inicial
Entre otras opciones posibles, se importa el paquete Jaal para visualizar interactivamente el grafo (en pestaña aparte). Descomente la primera línea, en caso de que no haya sido previamente instalado.

In [ ]:
#!pip install Jaal
from jaal import Jaal
from dash import html
import dash_html_components as html

Este paquete require un dataframe con al menos dos columnas ("from", "to") con los pares de nodos relacionados. La siguiente función genera ese dataframe desde el grafo RDF, mostrando sólo la parte final del URIRef de cada entidad.

In [ ]:
def rdf2edge_df(grafo):
    ternas = []
    for s, p, o in grafo:
        if isinstance(o,URIRef):
            oout = o.fragment
        else:
            oout = o
        ternas.append([s.fragment,p.fragment,oout])
    return pd.DataFrame(ternas,columns=["from","label","to"])

edge_df = rdf2edge_df(resp_grafo)
display(edge_df)

In [ ]:
edge_df.info()

In [ ]:
Jaal(edge_df).plot(directed=True)

## Consultas y actualización
RDFLib permite consultar y administrar el grafo de dos formas complementarias:
+ mediante el lenguaje estándar de consulta y administración SPARQL
+ mediante gestión directa y a bajo nivel de los objetos Python del grafo
### Consultas mediante SPARQL
Consulta SELECT: devuelve todas las coincidencias en el grafo del patrón declarado en WHERE. En este caso, cada persona de la que conste tanto su madre como su padre.

In [ ]:
consulta1='''
  PREFIX rec: <http://example.org/recursos#>
  PREFIX prop: <http://example.org/propiedades#>
  SELECT ?s ?m ?p
  WHERE {
    ?s prop:tiene_madre ?m ;
       prop:tiene_padre ?p .
  }
'''
resultados1 = g.query(consulta1)

for fila in resultados1:
    print(f"{fila.s}, madre: {fila.m}, padre: {fila.p}")

### Actualización mediante SPARQL
Actualización (UPDATE) del grafo: permite ejecutar INSERT o DELETE sobre entidades o tripletas seleccionadas en WHERE. En este caso, para cada instancia (?s,?m,?p) en el grafo del patrón en WHERE se asigna a cada una de sus componentes la clase Persona.

In [ ]:
consulta2='''
  PREFIX rec: <http://example.org/recursos#>
  PREFIX prop: <http://example.org/propiedades#>
  INSERT {
    rec:Persona rdf:type rdfs:Class .
    ?s rdf:type rec:Persona .
    ?m rdf:type rec:Persona .
    ?p rdf:type rec:Persona .
  }
  WHERE {
    ?s prop:tiene_madre ?m ;
       prop:tiene_padre ?p .
  }
'''
g.update(consulta2)

#print(g.serialize(format='turtle'))
g.serialize(destination='g_ttl.ttl', format='turtle')

In [ ]:
g_ttl

In [ ]:
import io
r_io = io.StringIO(g.serialize(format='turtle'))
print(type(r_io))

### Consulta mediante métodos Python
En este caso, se requieren todas las tripletas del grafo que tienen por propiedad 'lugar de nacimiento'.

In [ ]:
for s, p, o in g.triples( (None, EXP.lugar_nacimiento, None) ):
    print(s,p,o)

### Actualización mediante métodos Python
Se añade una terna, que declara la clase Ciudad. Y se inserta, para cada ciudad del grafo, su declaración como instancia de la clase Ciudad.

In [ ]:
g.add( (EXR.Ciudad, RDF.type, RDFS.Class) )

for s, p, o in g.triples( (None, EXP.lugar_nacimiento, None) ):
    g.add( (o, RDF.type, EXR.Ciudad) )

In [ ]:
# Comprobación: instancias de la clase Ciudad
for s, p, o in g.triples( (None, RDF.type, EXR.Ciudad) ):
    print(s,p,o)

In [ ]:
gio = io.StringIO()
gio.write(g_ttl)
print(type(gio))

In [ ]:
giob = io.BytesIO()
giob.write(g_ttl.encode('utf-8'))
print(type(giob))

In [ ]:
g_ttl_e = g.serialize(format="turtle", encoding="utf-8")
type(g_ttl_e)

In [ ]:
jlj = open('g_ttl.ttl','rb')
type (jlj)

In [ ]:
# FUNCIONA
import requests

headers = {
  'Content-Type': 'application/x-turtle',
  'Accept': 'application/json'
}

with open('g_ttl.ttl', 'rb') as f:
  requests.post('http://localhost:7200/repositories/parentesco/statements', data=f, headers=headers)

In [ ]:
# FUNCIONA
# requiere que g_ttl_e se haya serializado con encoding='utf-8' para generar bytes
headers = {
  'Content-Type': 'application/x-turtle',
  'Accept': 'application/json'
}

requests.post('http://localhost:7200/repositories/parentesco2/statements', data=g_ttl_e, headers=headers)

In [ ]:
#graph.serialize(my_url, format='application/rdf+xml')
#import requests
#import io
import sys

headers = {
  'Content-Type': 'application/x+turtle',
  'Accept': 'application/json'
}

requests.post('http://localhost:7200/repositories/parentesco/statements', data=r_io, headers=headers)

In [ ]:
!curl --help